In [ ]:
# Import libraries
import os
import numpy as np
import pandas as pd
from itertools import product

from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import make_scorer
from keras.wrappers.scikit_learn import KerasRegressor
from keras.callbacks import ReduceLROnPlateau
from keras import optimizers

from utils import load_image, get_centers, coord_transfm, draw_roi, mirror
from simplecnn import SimpleCNN
from resnet50 import ResNet50, Eucl_distance_tensor, Eucl_distance
from model_tuning import plot_history, plot_roi_centers

% matplotlib inline

In [ ]:
# Define global parameters

IMG_WIDTH = 32
IMG_HEIGHT = 32
IMG_CHANNELS = 1
TRAIN_SIZE = 5000
TRAIN_PATH = '../input/bone-lab/trainset/'
TEST_PATH = '../input/bone-lab/testset/'
ROI_PATH = "../input/bone-lab/roi/"

### Load and pre-process data/images

In [ ]:
# Read and load ROI data
df_centers_org = get_centers(ROI_PATH).sort_values(by='img_id') \
                                      .reset_index(drop=True)
print(df_centers_org.head())
df_centers = coord_transfm(df_centers_org)
print(df_centers.head())

In [ ]:
# Check every image matches the ROI file
images = pd.Series(sorted(os.listdir(TRAIN_PATH)))
img_ids = images.str.split('.').str[0]
assert df_centers.img_id.equals(img_ids), "Image lists don't match"

In [ ]:
# Randomly pick training samples from the trainset
train_images = images.sample(TRAIN_SIZE, random_state=10)
train_img_ids = train_images.str.split('.').str[0]

# Load the selected images and reduce resolution
mat_images = load_image(TRAIN_PATH, train_images, IMG_HEIGHT, IMG_WIDTH)

# Store all images into a dataframe
df_train = pd.DataFrame(mat_images, 
                        columns=['pxl' + str(i) for i in range(mat_images.shape[1])])
df_train.insert(0, 'img_id', train_img_ids.values)
df_train = pd.merge(df_train, df_centers[['img_id', 'cx', 'cy']], 
                    on='img_id', validate="1:1")
print(df_train.head())
print(df_train.shape)

### Manual data augmentation

In [ ]:
# Flip all train images around horizontal direction
df_train_hflip = mirror(df_train, 'h', IMG_HEIGHT, IMG_WIDTH)

# Flip all train images around horizontal direction
df_train_vflip = mirror(df_train, 'v', IMG_HEIGHT, IMG_WIDTH)

In [ ]:
# Merge original and reproduced images
df_train = pd.concat([df_train, df_train_hflip, df_train_vflip],
                     ignore_index=True)
print("New dataframe's shape: {}".format(df_train.shape))
df_centers = df_train[['img_id', 'cx', 'cy']]

In [ ]:
# Convert all images and their ROI centers into Numpy ndarray
X = df_train.drop(columns=['img_id', 'cx', 'cy']) \
            .values.reshape((-1, IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS))
Y = df_train[['cx', 'cy']].values
IDs = df_train.img_id.values

# Normalization
X /= 255.0

In [ ]:
# Free RAM space
del df_train, df_train_hflip, df_train_vflip, mat_images

In [ ]:
# Split train and validation sets
X_train, X_val, Y_train, Y_val, IDs_train, IDs_val \
= train_test_split(X, Y, IDs, test_size=0.1, random_state=1)

print("Trainset shape: {}".format(X_train.shape))
print("Validateset shape: {}".format(X_val.shape))

In [ ]:
# Display some sample images with known ROI
select_disp = df_centers.iloc[:TRAIN_SIZE].sample(n=3)
img_list = select_disp['img_id'].values
true_centers = select_disp[['cx', 'cy']].values
draw_roi(TRAIN_PATH, img_list, true_centers)

### Baseline model: Simple CNN
**Architechture:** 

    Input -> ((Conv2D->relu) x 2 -> MaxPool2D) x 2 -> Flatten -> Dense x 2 -> Output

In [ ]:
# Create the model
simple_cnn = SimpleCNN(INPUT_SHAPE=(IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS))
simple_cnn.summary()

In [ ]:
# Fit the model with learning rate to be reduced when no progress
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.9,
                              patience=5, min_lr=0.0001)
hist_cnn = simple_cnn.fit(X_train, Y_train, batch_size=64, epochs=100,
                          validation_split=0.2, callbacks=[reduce_lr])

In [ ]:
# Evaluate the model with validation set
cnn_scores_train = simple_cnn.evaluate(X_train, Y_train)
print("Score on trainset: {}".format(cnn_scores_train))

cnn_scores_val = simple_cnn.evaluate(X_val, Y_val)
print("Score on validate set: {}".format(cnn_scores_val))

In [ ]:
cnn_pred = simple_cnn.predict(X_val, verbose=True)
cnn_results = pd.DataFrame(np.concatenate([cnn_pred, Y_val], axis=1), 
                           columns = ['cx_pred', 'cy_pred', 'cx', 'cy'])
cnn_results.insert(0, 'img_id', IDs_val)
print(cnn_results.head())

In [ ]:
# Visualize some results from the first model
orig = [img_id for img_id in cnn_results.img_id 
        if 'f' not in img_id]
orig_imgs = cnn_results[cnn_results.img_id.isin(orig)]
selt_imgs = orig_imgs.sample(n=3)
img_list = selt_imgs['img_id'].values
true_centers = selt_imgs[['cx', 'cy']].values
pred_centers = selt_imgs[['cx_pred', 'cy_pred']].values

draw_roi(TRAIN_PATH, img_list, true_centers, 
         pred_centers, rows=2, cols=3, model_name="Simple CNN")

In [ ]:
# Plot the ROI centers that human determined (red) along with that model detected
selt_dots = orig_imgs.sample(n=15)
img_list = selt_dots['img_id'].values
true_centers = (selt_dots[['cx', 'cy']].values * 1330).astype(int)
pred_centers = (selt_dots[['cx_pred', 'cy_pred']].values * 1330).astype(int)

plot_roi_centers(true_centers, pred_centers, "Simple CNN")

In [ ]:
# Save the results of predictions
cnn_results.to_csv("cnn_results.csv")

# Save the model into HDF5 file
simple_cnn.save("simple_cnn.h5")

In [ ]:
# Plot training history on loss
plot_history(hist_cnn, 'Simple CNN', xlim=(0,60), ylim=(0.0, 0.03))

### Second model: ResNet-50
**Architecture**

    INPUT -> CONV2D -> BATCHNORM -> RELU -> MAXPOOL -> CONVBLOCK -> IDBLOCK x 2 -> CONVBLOCK -> 
    IDBLOCK x 3 -> CONVBLOCK -> IDBLOCK x 5 -> CONVBLOCK -> IDBLOCK x 2 -> AVGPOOL -> FC x 2 -> 
    Dropout -> Output



In [ ]:
# Create the model instance
resnet = ResNet50(input_shape=(IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS),
                  lr_power=-2.5,
                  extra_layers=(1024, 256, 32),
                  dropouts=(0., 0., 0.))
resnet.summary()

In [ ]:
# Fit the model with learning rate to be reduced when no progress
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.9,
                              patience=5, min_lr=0.0001)
hist_resnet = resnet.fit(X_train, Y_train, batch_size=32, epochs=120,
                         validation_split=0.2, callbacks=[reduce_lr])

In [ ]:
# Evaluate the model with validation set
ResNet_scores_train = resnet.evaluate(X_train, Y_train, batch_size=512)
print("Score on trainset: {}".format(ResNet_scores_train))

ResNet_scores_val = resnet.evaluate(X_val, Y_val)
print("Score on validate set: {}".format(ResNet_scores_val))

In [ ]:
ResNet_pred = resnet.predict(X_val, verbose=True)
ResNet_results = pd.DataFrame(np.concatenate([ResNet_pred, Y_val], axis=1), 
                              columns = ['cx_pred', 'cy_pred', 'cx', 'cy'])
ResNet_results.insert(0, 'img_id', IDs_val)
print(ResNet_results.head())

In [ ]:
# Visualize some results from the first model
orig = [img_id for img_id in ResNet_results.img_id 
        if 'f' not in img_id]
orig_imgs = ResNet_results[ResNet_results.img_id.isin(orig)]
selt_imgs = orig_imgs.sample(n=3)
img_list = selt_imgs['img_id'].values
true_centers = selt_imgs[['cx', 'cy']].values
pred_centers = selt_imgs[['cx_pred', 'cy_pred']].values

draw_roi(TRAIN_PATH, img_list, true_centers, pred_centers, 
         rows=2, cols=3, model_name="ResNet50")

In [ ]:
# Plot the ROI centers that human determined (red) along with that model detected
selt_dots = orig_imgs.sample(n=10)
img_list = selt_dots['img_id'].values
true_centers = (selt_dots[['cx', 'cy']].values * 1330).astype(int)
pred_centers = (selt_dots[['cx_pred', 'cy_pred']].values * 1330).astype(int)

plot_roi_centers(true_centers, pred_centers, "ResNet-50")

In [ ]:
# Save the results
ResNet_results.to_csv("ResNet_results.csv")

# Save the model into HDF5 file
resnet.save("ResNet50_BoneCT.h5")

In [ ]:
# Plot training history on loss
plot_history(hist_resnet, 'ResNet-50', xlim=(10, 120), ylim=(0.0, 0.08))

### Hyper-parameters tunning for ResNet-50

** Grid search for best hyper-parameters **

In [ ]:
# Conduct randomized grid search to find appropriate hyper-parameters for the ResNet-50 model

resnet = KerasRegressor(build_fn = ResNet50, 
                        input_shape = (IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS),
                        epochs = 50,
                        verbose = True)

lr_power = stats.uniform(loc=-3.3, scale=1.3)
batch_size = np.power(2, range(3, 11))
dense_1 = [1024, 512, 256]
dense_2 = [256, 128, 56]
dense_3 = [56, 32, 16]
extra_layers = list(product(dense_1, dense_2, dense_3))
dropouts = list(product([0.5, 0.25, 0.0], repeat=3))

hparam_dist = {'lr_power': lr_power,
               'batch_size': batch_size,
               'extra_layers': extra_layers,
               'dropouts': dropouts
              }

hparam_search = RandomizedSearchCV(estimator=resnet,
                                   param_distributions=hparam_dist,
                                   n_iter=16, cv=2, verbose=True)

hparam_search.fit(X_train, Y_train)

In [ ]:
# Show the results

print("Best mean loss: {:.5f} with\n {}".format(-hparam_search.best_score_, hparam_search.best_params_))
means = -hparam_search.cv_results_['mean_test_score']
stds = hparam_search.cv_results_['std_test_score']
params = hparam_search.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("___" * 10)
    print("%.5f (%.3f) with\n %r" % (mean, stdev, param))